In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/learn-ai-bbc/BBC News Train.csv
/kaggle/input/learn-ai-bbc/BBC News Sample Solution.csv
/kaggle/input/learn-ai-bbc/BBC News Test.csv


Objectives BBC classification: Classification assisted with unsupervised learning

1. EDA

2. TF-IDF

3. SVD

4. classification algorithm

Preprocessing with a tf id vectorizer is required due to the data being text. 

In [2]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, MiniBatchNMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

Part 1: use 

In [3]:
train_data = pd.read_csv('/kaggle/input/learn-ai-bbc/BBC News Train.csv')
test_data = pd.read_csv('/kaggle/input/learn-ai-bbc/BBC News Test.csv')

In [4]:
print("Train data shape:", train_data.shape)
print("Test data shape:", test_data.shape)
print("\nTrain data preview:")
print(train_data.head())
print("\nCategories in training data:")
print(train_data['Category'].value_counts())

# Prepare training data
x_train = train_data['Text']
y_train = train_data['Category']


Train data shape: (1490, 3)
Test data shape: (735, 2)

Train data preview:
   ArticleId                                               Text  Category
0       1833  worldcom ex-boss launches defence lawyers defe...  business
1        154  german business confidence slides german busin...  business
2       1101  bbc poll indicates economic gloom citizens in ...  business
3       1976  lifestyle  governs mobile choice  faster  bett...      tech
4        917  enron bosses in $168m payout eighteen former e...  business

Categories in training data:
Category
sport            346
business         336
politics         274
entertainment    273
tech             261
Name: count, dtype: int64


Preprocessing

In [5]:
y = train_data['Category']
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

y = le.fit_transform(y)

In [6]:
print(np.unique(y))

[0 1 2 3 4]


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error
vectorizer = TfidfVectorizer()


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
x_test = test_data['Text']
x = x_train
print(train_data.head(1))
vectorizer = TfidfVectorizer()

tfidf = vectorizer.fit(x)
tfid = vectorizer.transform(x)
#X_test = vectorizer.fit_transform(x_test)
#X_test = X_test.toarray()

nmf = NMF(
    n_components=20,
    random_state=1,
    beta_loss="frobenius",
    alpha_W=0.00005,
    alpha_H=0.00005,
    l1_ratio=1,
).fit(tfid)

nmfd = nmf.transform(tfid)

   ArticleId                                               Text  Category
0       1833  worldcom ex-boss launches defence lawyers defe...  business


/usr/local/lib/python3.11/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


EDA. Latent topic detection

In [9]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, MiniBatchNMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#EDA

lda = LatentDirichletAllocation(
    n_components=5,   # Try different numbers
    random_state=42,
    learning_method='batch'  # Or 'online'
)

lda.fit(tfid)

def display_topics(model, feature_names, no_top_words=10):
    for idx, topic in enumerate(model.components_):
        print(f"Topic {idx}: ", end='')
        print(" | ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

feature_names = vectorizer.get_feature_names_out()
display_topics(lda, feature_names)


Topic 0: the | to | of | and | in | for | is | that | it | he
Topic 1: fiat | borussia | dortmund | godzilla | elgindy | scoggins | ferrari | maserati | rigoglioso | mcindoe
Topic 2: boerse | cairn | euronext | pernod | inbev | southcorp | holcim | gw | lacroix | cement
Topic 3: uwb | skype | capriati | ivanovic | nestle | carmen | printer | treviso | broadreach | cartridges
Topic 4: ssl | soderling | savoy | fairmont | carib | cotonsport | condom | strand | alwaleed | cannibals


In [10]:
def plot_top_words(model, feature_names, n_top_words, title):
    fig, axes = plt.subplots(2, 5, figsize=(30, 15), sharex=True)
    axes = axes.flatten()
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[-n_top_words:]
        top_features = feature_names[top_features_ind]
        weights = topic[top_features_ind]

        ax = axes[topic_idx]
        ax.barh(top_features, weights, height=0.7)
        ax.set_title(f"Topic {topic_idx + 1}", fontdict={"fontsize": 30})
        ax.tick_params(axis="both", which="major", labelsize=20)
        for i in "top right left".split():
            ax.spines[i].set_visible(False)
        fig.suptitle(title, fontsize=40)

    plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)
    plt.show()

In [11]:
def display_topics(model, feature_names, no_top_words=10):
    for idx, topic in enumerate(model.components_):
        print(f"Topic {idx}: ", end='')
        print(" | ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

feature_names = vectorizer.get_feature_names_out()
display_topics(lda, feature_names)


Topic 0: the | to | of | and | in | for | is | that | it | he
Topic 1: fiat | borussia | dortmund | godzilla | elgindy | scoggins | ferrari | maserati | rigoglioso | mcindoe
Topic 2: boerse | cairn | euronext | pernod | inbev | southcorp | holcim | gw | lacroix | cement
Topic 3: uwb | skype | capriati | ivanovic | nestle | carmen | printer | treviso | broadreach | cartridges
Topic 4: ssl | soderling | savoy | fairmont | carib | cotonsport | condom | strand | alwaleed | cannibals


In [12]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
X_train, X_test, y_train, y_test = train_test_split(nmfd,y, test_size=0.2)
clf = GaussianNB()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(y_pred)


[[15  0 16 42  5]
 [10  0  4 26  0]
 [ 8  0 33  7  5]
 [18  0 17 39  3]
 [ 7  0 17 17  9]]
[3 3 0 2 2 3 2 3 3 3 0 3 2 3 3 0 2 3 3 2 3 2 0 0 2 3 2 3 3 3 2 2 0 0 3 0 2
 2 2 2 3 0 0 2 2 3 3 4 3 3 2 3 3 4 2 3 2 2 2 0 0 0 2 0 0 2 3 4 3 2 2 0 4 3
 2 3 0 4 3 2 3 3 2 3 2 2 3 3 0 3 3 2 3 3 0 4 3 3 0 3 3 3 4 3 0 3 4 4 3 0 3
 2 2 3 2 4 0 2 3 3 0 2 2 3 2 2 2 4 4 2 3 3 2 3 2 4 3 2 3 2 0 2 3 0 4 3 3 3
 3 3 0 0 0 2 0 3 2 3 3 3 3 2 3 3 2 3 2 3 2 3 0 3 0 3 3 3 0 3 0 3 3 3 3 3 3
 2 2 3 2 3 2 0 2 0 3 2 3 2 3 2 0 0 4 0 2 0 3 0 0 0 3 2 3 3 0 3 3 3 3 4 4 3
 0 2 0 2 3 3 3 2 2 4 0 4 2 2 2 2 0 3 2 3 3 4 3 3 3 2 2 2 0 3 3 3 4 2 3 4 0
 3 2 3 2 0 2 3 3 2 3 3 3 2 3 3 3 3 0 3 3 2 3 0 2 3 0 3 0 2 0 0 3 2 3 2 2 3
 0 3]


In [13]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)

print(rmse)

1.9019594276507399


In [14]:
print(y_pred.shape)

(298,)


In [15]:
#doing true predictions with nmf

X_test_tfidf = vectorizer.transform(test_data['Text'])

X_test_nmf = nmf.fit_transform(X_test_tfidf)

preds = clf.predict(X_test_nmf)

print(preds)

[3 0 3 4 3 4 2 4 3 3 3 2 4 3 3 0 2 4 3 3 2 2 2 3 3 3 4 3 0 0 2 0 2 3 3 2 0
 3 3 3 3 0 2 4 2 2 3 3 2 0 3 0 4 2 3 2 2 0 2 2 3 0 3 0 4 3 3 2 2 2 3 2 2 2
 4 3 2 2 2 4 2 0 4 2 0 2 4 0 2 2 2 4 3 3 3 3 0 3 3 3 0 4 2 0 4 3 2 3 3 3 3
 0 3 2 0 2 2 3 2 3 3 3 4 0 2 3 0 2 3 0 3 2 2 3 0 3 2 4 3 3 2 2 2 0 0 4 4 4
 3 2 2 2 2 3 2 3 3 3 0 2 2 2 2 0 0 2 2 0 2 2 0 0 3 2 3 2 3 3 3 2 3 4 0 4 0
 4 2 0 4 0 2 3 0 2 2 0 2 0 2 0 4 2 4 4 0 0 3 2 3 0 3 3 3 3 3 2 3 3 2 4 3 2
 4 2 2 0 3 2 0 0 0 0 2 2 2 2 0 4 2 0 3 2 3 2 3 3 2 3 0 0 3 3 2 0 0 0 3 4 3
 2 3 2 3 2 3 4 2 2 0 2 2 4 0 0 0 0 2 3 2 2 4 3 2 0 0 0 2 3 2 2 0 0 0 2 2 3
 3 2 3 3 0 0 0 3 0 2 0 2 3 2 4 3 2 4 2 2 2 2 2 3 2 3 2 2 3 2 2 2 2 3 4 3 3
 0 2 3 2 2 0 0 0 4 2 0 2 0 0 3 2 2 3 3 0 2 0 2 3 0 3 2 2 2 2 2 2 0 2 4 2 2
 3 3 3 3 3 3 2 0 2 0 2 2 2 3 3 3 2 2 3 2 4 2 3 0 4 0 3 3 0 0 3 2 3 4 2 2 3
 3 4 0 2 2 3 4 2 2 2 3 4 3 2 3 2 0 0 2 2 4 3 4 2 3 0 3 2 3 0 2 3 0 2 3 3 3
 0 0 2 2 0 4 2 2 3 0 2 2 3 0 0 0 3 2 2 0 3 0 3 3 2 0 0 0 2 2 2 2 3 0 3 3 2
 3 4 3 3 2 3 0 2 4 2 0 2 

/usr/local/lib/python3.11/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [16]:
print(X_test_nmf.shape)

(735, 20)


In [17]:
print(test_data.shape)

print(X_test_tfidf.shape)

(735, 2)
(735, 24746)


In [18]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
X_train, X_test, y_train, y_test = train_test_split(tfid.toarray(),y, test_size=0.2)
clf = GaussianNB()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(y_pred)

[[57  0  1  0  4]
 [ 2 47  1  1  2]
 [ 5  1 49  0  1]
 [ 1  0  0 76  0]
 [ 2  3  0  0 45]]
[1 3 1 1 0 3 0 3 4 3 2 2 0 1 2 2 1 2 1 4 4 3 1 1 3 4 3 1 0 0 1 3 1 3 2 1 0
 3 3 3 0 1 0 0 3 3 3 4 4 0 0 2 0 2 1 2 0 1 1 0 0 0 0 4 4 3 2 0 2 1 4 0 2 4
 1 0 3 0 2 1 3 0 0 1 0 2 4 1 2 3 0 4 2 0 1 1 3 1 0 0 0 2 4 2 0 2 3 3 1 4 1
 4 0 0 4 3 2 4 4 4 2 0 3 4 1 2 3 3 3 0 3 3 2 3 0 2 4 4 0 1 2 4 4 3 4 1 1 4
 1 1 1 0 2 0 0 3 3 2 4 3 0 0 2 1 4 4 3 4 2 1 2 3 2 1 4 0 3 1 3 0 3 0 3 1 0
 1 3 3 3 0 4 3 1 3 3 3 3 0 3 4 3 3 4 3 3 1 3 3 0 1 2 0 2 3 0 0 3 2 2 3 1 3
 2 0 3 2 4 4 2 1 4 0 1 4 4 3 0 3 4 2 0 4 3 2 3 4 1 4 0 2 0 0 4 4 3 4 2 4 3
 3 4 2 0 0 0 2 3 2 0 2 1 0 0 4 3 1 2 1 2 4 2 4 3 0 4 3 1 1 3 0 0 3 2 3 3 3
 2 3]


In [19]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)

print(rmse)

0.7900386533743164


In [20]:
print("\nCreating submission file...")
submission = pd.DataFrame({
    'ArticleId': test_data['ArticleId'],
    'Category': y_test_pred_categories
})


Creating submission file...


NameError: name 'y_test_pred_categories' is not defined

Conclusion: preprocessing with NMF has failed to increase the accuracy 

In [ ]:
#last TODOs 